# WEATHER QUERY

imports, libraries, and options

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.cloud import bigquery
from google.oauth2 import service_account

pd.options.display.max_colwidth = 500

<br>
read in appellations df from pkl as stations

In [2]:
stations= pd.read_pickle('datasets/appellations.pkl')

# length of stations: 138
stations.shape[0]

138

<br>
use own json API file from Google Cloud BigQuery to allow access to the API from notebook

In [4]:
cred= service_account.Credentials.from_service_account_file('OWN API HERE')

client= bigquery.Client(credentials= cred, project= cred.project_id)

<br>
make weather_stations df of weather stations within 0.2° lat and 0.2° long (~ 22 km) of appellation lat and long using the GHCN daily dataset

In [ ]:
# https://github.com/RoaldSchuring/studying_terroir/blob/master/Investigating%20Terroir%20Data%20Prep.ipynb
# https://github.com/polong-lin/london-weather-bigquery/blob/master/London%20Weather%20Data%20-%20Python.ipynb
# https://www.w3schools.com/python/ref_string_format.asp

weather_station_list= []
for i in range(stations.shape[0]):
    station= """SELECT
      id,
      name,
      latitude,
      longitude,
    FROM
      `bigquery-public-data.ghcn_d.ghcnd_stations`
    WHERE
      latitude > {}
      AND latitude < {}
      AND longitude > {}
      AND longitude < {}
    LIMIT 1
    """.format(stations['lat'][i] - 0.2, stations['lat'][i] + 0.2, stations['long'][i] - 0.2, stations['long'][i]+ 0.2)
    
    closest_station = client.query(station).to_dataframe()
    weather_station_list.append(closest_station)

# concatentate all items in weather_station_list to weather_stations df
weather_stations= pd.concat(weather_station_list)
weather_stations.reset_index(inplace= True, drop= True)

# confirm weather_stations has the same df length as stations: 138
weather_stations.shape[0]

<br>
make new df only of weather station id and appllation and merge to station df on index

In [98]:
weather_stations_id= weather_stations[['id']]

stations_id= stations.merge(right= weather_stations_id, how= 'left', left_index= True, right_index= True)
stations_id.drop(columns= ['location', 'geocode', 'lat', 'long'], inplace= True)

stations_id.head()

<br>
read in project df and merge with stations df on appellation

In [99]:
project= pd.read_pickle('datasets/project.pkl')
project= project.merge(right= stations_id, how= 'left', on= 'appellation')

<br>
make new vintage_loc df containing all appellations and their prospective years

In [206]:
vintage_loc= project.copy()[['id', 'vintage']]

# drop duplicates and reset index
vintage_loc.drop_duplicates(inplace= True)
vintage_loc.reset_index(inplace= True, drop= True)

In [207]:
vintage_loc

,id,vintage
0,USC00046730,2000
1,USC00046730,2015
2,USC00046730,2013
3,USC00046730,2012
4,USC00046730,2001
...,...,...
1477,US1CALA0007,2009
1478,USC00047876,2019
1479,USW00023240,2005
1480,USC00045296,2003


<br>
make climate_stations df of the average precipitation, minimum temperature, maximum temperature, average temperature minimum, and average temperature maximum for each month for each vintage for each appellation from the GHCN daily dataset

In [216]:
# https://github.com/RoaldSchuring/studying_terroir/blob/master/Investigating%20Terroir%20Data%20Prep.ipynb
# https://github.com/polong-lin/london-weather-bigquery/blob/master/London%20Weather%20Data%20-%20Python.ipynb
# https://www.w3schools.com/python/ref_string_format.asp

climate_list= []
for i in range(vintage_loc_1_200.shape[0]):
    climate= '''SELECT
      id,
      EXTRACT(YEAR FROM date) AS vintage,
      EXTRACT(MONTH FROM date) AS calendar_month,
      AVG(prcp) as prcp,
      MIN(tmin) as tmin,
      MAX(tmax) as tmax,
      AVG(tmin) as avg_tmin,
      AVG(tmax) as avg_tmax
    FROM (
        SELECT
            id,
            date,
            IF (element = 'PRCP', value/10, NULL) AS prcp,
            IF (element = 'TMIN', value/10, NULL) AS tmin,
            IF (element = 'TMAX', value/10, NULL) AS tmax,

        FROM
            `bigquery-public-data.ghcn_d.ghcnd_{}`
        WHERE 
            id = '{}'
            AND qflag IS NULL
    )
    GROUP BY
      id, calendar_month, vintage

    '''.format(vintage_loc_1_200['vintage'][i] , vintage_loc_1_200['id'][i] )

    station_climate = client.query(climate).to_dataframe()
    
    station_climate= station_climate[(station_climate['calendar_month'] > 2) & (station_climate['calendar_month'] < 11)].sort_values(
    by= 'calendar_month')
    
    for month in station_climate['calendar_month']:
        station_climate[f'{month}_prcp']= station_climate.loc[station_climate['calendar_month'] == month, 'prcp'].iloc[0]
        station_climate[f'{month}_tmin']= station_climate.loc[station_climate['calendar_month'] == month, 'tmin'].iloc[0]
        station_climate[f'{month}_tmax']= station_climate.loc[station_climate['calendar_month'] == month, 'tmax'].iloc[0]
        station_climate[f'{month}_avg_tmin']= station_climate.loc[station_climate['calendar_month'] == month, 'avg_tmin'].iloc[0]
        station_climate[f'{month}_avg_tmax']= station_climate.loc[station_climate['calendar_month'] == month, 'avg_tmax'].iloc[0]
    
    station_climate.drop(columns= ['calendar_month', 'prcp', 'tmin', 'tmax', 'avg_tmin', 'avg_tmax'], inplace= True)
    yearly_climate= station_climate[:1]
    climate_list.append(yearly_climate)

# concatenate each list as a rows to the climate_stations df
climate_stations= pd.concat(climate_list)
climate_stations.reset_index(inplace= True, drop= True)


# check length of df: 441 of 1482 stations were returned
climate_stations.shape[0]

In [199]:
climate_stations

,id,vintage,3_prcp,3_tmin,3_tmax,3_avg_tmin,3_avg_tmax,4_prcp,4_tmin,4_tmax,...,9_prcp,9_tmin,9_tmax,9_avg_tmin,9_avg_tmax,10_prcp,10_tmin,10_tmax,10_avg_tmin,10_avg_tmax
0,USC00046730,2000,1.270968,0.6,25.0,4.138710,18.496774,1.320000,2.2,29.4,...,0.027586,5.6,38.9,9.779310,29.448276,1.100000,1.7,35.6,6.361290,23.090323
1,USC00046730,2015,0.083871,0.0,32.8,4.535484,25.864516,0.313333,1.1,33.9,...,0.043333,6.7,40.6,11.263333,33.023333,0.061290,5.0,38.3,10.041935,29.658065
2,USC00046730,2013,0.567742,-0.6,31.1,4.164516,23.477419,0.060000,0.0,35.6,...,0.000000,3.9,38.9,10.836667,31.906667,0.009677,2.2,32.8,4.216129,27.600000
3,USC00046730,2012,1.996774,-2.8,30.0,2.145161,19.387097,2.200000,-0.6,36.1,...,0.000000,6.7,39.4,10.163333,33.703333,0.240000,1.7,40.6,7.970000,27.336667
4,USC00046730,2001,2.948387,1.1,27.8,5.745161,19.464516,0.573333,-0.6,30.0,...,0.000000,6.7,35.6,9.010000,30.426667,0.196774,3.3,38.3,6.835484,27.558065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,USR0000CPIK,2000,NaN,-0.6,57.2,3.803226,16.303226,NaN,1.7,27.2,...,NaN,7.2,34.4,14.267857,26.521429,NaN,2.2,31.1,9.138710,18.619355
437,USR0000CPIK,1998,NaN,-2.2,20.0,3.735484,11.616129,NaN,-1.1,24.4,...,NaN,7.2,35.0,15.463333,24.390000,NaN,4.4,24.4,9.383871,18.196774
438,USR0000CPIK,2005,NaN,-0.6,26.1,4.693548,14.635484,NaN,-1.1,22.8,...,NaN,7.8,32.8,13.040000,25.693333,NaN,3.9,29.4,9.277419,21.145161
439,US1CALA0007,2012,2.287097,NaN,NaN,NaN,NaN,2.785185,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.714286,NaN,NaN,NaN,NaN


In [ ]:
climate_stations.isnull().sum()

id               0
vintage          0
3_prcp          27
3_tmin         194
3_tmax         194
3_avg_tmin     194
3_avg_tmax     194
4_prcp          33
4_tmin         191
4_tmax         191
4_avg_tmin     191
4_avg_tmax     191
5_prcp          37
5_tmin         190
5_tmax         190
5_avg_tmin     190
5_avg_tmax     190
6_prcp          46
6_tmin         190
6_tmax         190
6_avg_tmin     190
6_avg_tmax     190
7_prcp          56
7_tmin         190
7_tmax         191
7_avg_tmin     190
7_avg_tmax     191
8_prcp          54
8_tmin         190
8_tmax         190
8_avg_tmin     190
8_avg_tmax     190
9_prcp          48
9_tmin         193
9_tmax         193
9_avg_tmin     193
9_avg_tmax     193
10_prcp         33
10_tmin        190
10_tmax        190
10_avg_tmin    190
10_avg_tmax    190
dtype: int64

<br>
make climate df to merge project df and climate stations df using an inner join and then drop all rows with nulls in the climate columns

In [244]:
climate= project.merge(right= climate_stations, how= 'inner', left_on= ['id', 'vintage'], right_on= ['id', 'vintage']).dropna(subset= ['3_prcp', '3_tmin', 
        '3_tmax', '3_avg_tmin', '3_avg_tmax', '4_prcp', '4_tmin', '4_tmax', '4_avg_tmin', '4_avg_tmax', '5_prcp', '5_tmin', '5_tmax', '5_avg_tmin', '5_avg_tmax', 
        '6_prcp', '6_tmin', '6_tmax', '6_avg_tmin', '6_avg_tmax', '7_prcp', '7_tmin', '7_tmax', '7_avg_tmin', '7_avg_tmax', '8_prcp', '8_tmin', '8_tmax', 
        '8_avg_tmin', '8_avg_tmax', '9_prcp', '9_tmin', '9_tmax', '9_avg_tmin', '9_avg_tmax', '10_prcp', '10_tmin', '10_tmax', '10_avg_tmin', '10_avg_tmax']).reset_index(drop= True)

In [245]:
climate

,wine,name,vintage,category,varietal,winery,appellation,designation,alcohol,price,...,9_prcp,9_tmin,9_tmax,9_avg_tmin,9_avg_tmax,10_prcp,10_tmin,10_tmax,10_avg_tmin,10_avg_tmax
0,J. Lohr 2000 Hilltop Vineyard Cabernet Sauvignon (Paso Robles),Hilltop Vineyard Cabernet Sauvignon,2000,Red,Cabernet Sauvignon,J. Lohr,"Paso Robles, Central Coast, California, US",Hilltop Vineyard,NaN,$32,...,0.027586,5.6,38.9,9.779310,29.448276,1.100000,1.7,35.6,6.361290,23.090323
1,Château Potelle 2000 Zinfandel (Paso Robles),Zinfandel,2000,Red,Zinfandel,Château Potelle,"Paso Robles, Central Coast, California, US",NaN,NaN,$22,...,0.027586,5.6,38.9,9.779310,29.448276,1.100000,1.7,35.6,6.361290,23.090323
2,Midnight 2000 Capriccio Italien Cabernet Sauvignon-Sangiovese (Paso Robles),Capriccio Italien Cabernet Sauvignon-Sangiovese,2000,Red,"Cabernet Sauvignon-Sangiovese, Italian Red",Midnight,"Paso Robles, Central Coast, California, US",Capriccio Italien,14%,$30,...,0.027586,5.6,38.9,9.779310,29.448276,1.100000,1.7,35.6,6.361290,23.090323
3,J. Lohr 2000 PAU Red (Paso Robles),PAU Red,2000,Red,"Cabernet Blend, Bordeaux-style Red Blend",J. Lohr,"Paso Robles, Central Coast, California, US",PAU,NaN,$50,...,0.027586,5.6,38.9,9.779310,29.448276,1.100000,1.7,35.6,6.361290,23.090323
4,Robert Hall 2000 Cabernet Sauvignon (Paso Robles),Cabernet Sauvignon,2000,Red,Cabernet Sauvignon,Robert Hall,"Paso Robles, Central Coast, California, US",NaN,NaN,$18,...,0.027586,5.6,38.9,9.779310,29.448276,1.100000,1.7,35.6,6.361290,23.090323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14076,Calcareous 2006 Cabernet Sauvignon (York Mountain),Cabernet Sauvignon,2006,Red,Cabernet Sauvignon,Calcareous,"York Mountain, Central Coast, California, US",NaN,15.8%,$34,...,0.000000,10.0,32.2,11.746667,17.906667,0.080645,7.2,27.2,9.938710,19.154839
14077,Stephen's 2005 MacBride Vineyard Chardonnay (York Mountain),MacBride Vineyard Chardonnay,2005,White,Chardonnay,Stephen's,"York Mountain, Central Coast, California, US",MacBride Vineyard,14%,$28,...,0.070000,8.3,23.9,10.860000,17.270000,0.238710,7.2,30.6,10.074194,18.551613
14078,Calcareous 2005 Cabernet Sauvignon (York Mountain),Cabernet Sauvignon,2005,Red,Cabernet Sauvignon,Calcareous,"York Mountain, Central Coast, California, US",NaN,15.7%,$32,...,0.070000,8.3,23.9,10.860000,17.270000,0.238710,7.2,30.6,10.074194,18.551613
14079,Sonnet 2005 Kruse Vineyard Pinot Noir (York Mountain),Kruse Vineyard Pinot Noir,2005,Red,Pinot Noir,Sonnet,"York Mountain, Central Coast, California, US",Kruse Vineyard,14.5%,$40,...,0.070000,8.3,23.9,10.860000,17.270000,0.238710,7.2,30.6,10.074194,18.551613


<br>
save climate df to pkl

In [246]:
climate.to_pickle('datasets/climate.pkl')